In [ ]:
%%writefile ./pytorch_mnist_hvd.py

from __future__ import print_function
import argparse
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data.distributed
import horovod.torch as hvd

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=42, metavar='S',
                    help='random seed (default: 42)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--fp16-allreduce', action='store_true', default=False,
                    help='use fp16 compression during allreduce')
parser.add_argument('--use-adasum', action='store_true', default=False,
                    help='use adasum algorithm to do reduction')


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


def train(epoch):
    model.train()
    # Horovod: set epoch to sampler for shuffling.
    train_sampler.set_epoch(epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            # Horovod: use train_sampler to determine the number of examples in
            # this worker's partition.
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_sampler),
                100. * batch_idx / len(train_loader), loss.item()))


def metric_average(val, name):
    tensor = torch.tensor(val)
    avg_tensor = hvd.allreduce(tensor, name=name)
    return avg_tensor.item()


def test():
    model.eval()
    test_loss = 0.
    test_accuracy = 0.
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        test_accuracy += pred.eq(target.data.view_as(pred)).cpu().float().sum()

    # Horovod: use test_sampler to determine the number of examples in
    # this worker's partition.
    test_loss /= len(test_sampler)
    test_accuracy /= len(test_sampler)

    # Horovod: average metric values across workers.
    test_loss = metric_average(test_loss, 'avg_loss')
    test_accuracy = metric_average(test_accuracy, 'avg_accuracy')

    # Horovod: print output only on first rank.
    if hvd.rank() == 0:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(
            test_loss, 100. * test_accuracy))


if __name__ == '__main__':
    args = parser.parse_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    # Horovod: initialize library.
    hvd.init()
    torch.manual_seed(args.seed)

    if args.cuda:
        # Horovod: pin GPU to local rank.
        torch.cuda.set_device(hvd.local_rank())
        torch.cuda.manual_seed(args.seed)


    # Horovod: limit # of CPU threads to be used per worker.
    torch.set_num_threads(1)

    kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
    # When supported, use 'forkserver' to spawn dataloader workers instead of 'fork' to prevent
    # issues with Infiniband implementations that are not fork-safe
    if (kwargs.get('num_workers', 0) > 0 and hasattr(mp, '_supports_context') and
            mp._supports_context and 'forkserver' in mp.get_all_start_methods()):
        kwargs['multiprocessing_context'] = 'forkserver'

    train_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
    # Horovod: use DistributedSampler to partition the training data.
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, sampler=train_sampler, **kwargs)

    test_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]))
    # Horovod: use DistributedSampler to partition the test data.
    test_sampler = torch.utils.data.distributed.DistributedSampler(
        test_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size,
                                              sampler=test_sampler, **kwargs)

    model = Net()

    # By default, Adasum doesn't need scaling up learning rate.
    lr_scaler = hvd.size() if not args.use_adasum else 1

    if args.cuda:
        # Move model to GPU.
        model.cuda()
        # If using GPU Adasum allreduce, scale learning rate by local_size.
        if args.use_adasum and hvd.nccl_built():
            lr_scaler = hvd.local_size()

    # Horovod: scale learning rate by lr_scaler.
    optimizer = optim.SGD(model.parameters(), lr=args.lr * lr_scaler,
                          momentum=args.momentum)

    # Horovod: broadcast parameters & optimizer state.
    hvd.broadcast_parameters(model.state_dict(), root_rank=0)
    hvd.broadcast_optimizer_state(optimizer, root_rank=0)

    # Horovod: (optional) compression algorithm.
    compression = hvd.Compression.fp16 if args.fp16_allreduce else hvd.Compression.none

    # Horovod: wrap optimizer with DistributedOptimizer.
    optimizer = hvd.DistributedOptimizer(optimizer,
                                         named_parameters=model.named_parameters(),
                                         compression=compression,
                                         op=hvd.Adasum if args.use_adasum else hvd.Average)

    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test()

# Separate Notebook Codes Below

In [1]:
from __future__ import print_function
import argparse
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data.distributed
import horovod.torch as hvd

# Do Make Network

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

## Define training function

In [3]:
def train(epoch):
    model.train()
    ##### HOROVOD ##### --- train_sampler, optimizer wrapper
    # Horovod: set epoch to sampler for shuffling.
    train_sampler.set_epoch(epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            ##### HOROVOD ##### --- train_sampler
            # Horovod: use train_sampler to determine the number of examples in
            # this worker's partition.
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_sampler),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def metric_average(val, name):
    tensor = torch.tensor(val)
    ##### HOROVOD ##### -- allreduce (tensor average)
    avg_tensor = hvd.allreduce(tensor, name=name)    
    return avg_tensor.item()

# test function

In [5]:
def test():
    model.eval()
    test_loss = 0.
    test_accuracy = 0.
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        test_accuracy += pred.eq(target.data.view_as(pred)).cpu().float().sum()

    ##### HOROVOD #####
    # Horovod: use test_sampler to determine the number of examples in
    # this worker's partition.
    test_loss /= len(test_sampler)
    test_accuracy /= len(test_sampler)

    ##### HOROVOD #####
    # Horovod: average metric values across workers.
    test_loss = metric_average(test_loss, 'avg_loss')
    test_accuracy = metric_average(test_accuracy, 'avg_accuracy')

    # Horovod: print output only on first rank.
    ##### HOROVOD #####
    if hvd.rank() == 0:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(
            test_loss, 100. * test_accuracy))

# main function

In [10]:
if __name__ == '__main__':
    
    ##### TODO: This is arguments set #####
    ##### CUDA SETTING #####
    no_cuda = False
    cuda = not no_cuda and torch.cuda.is_available()
    #########################
    batch_size = 64
    test_batch_size = 128
    epochs = 10    
    momentum = 0.5
    lr = 0.01
    log_interval = 10
    # For Parallel/Distributed
    seed = 42
    use_adasum = False

    # Horovod: initialize library.
    ##### HOROVOD #####
    hvd.init()
    torch.manual_seed(seed)

    if cuda:
        # Horovod: pin GPU to local rank.
        ##### HOROVOD #####
        torch.cuda.set_device(hvd.local_rank())
        ##### TODO:Need argument #####
        #torch.cuda.manual_seed(args.seed)
        torch.cuda.manual_seed(seed)


    # Horovod: limit # of CPU threads to be used per worker.
    torch.set_num_threads(1)

    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
    # When supported, use 'forkserver' to spawn dataloader workers instead of 'fork' to prevent
    # issues with Infiniband implementations that are not fork-safe
    if (kwargs.get('num_workers', 0) > 0 and hasattr(mp, '_supports_context') and
            mp._supports_context and 'forkserver' in mp.get_all_start_methods()):
        kwargs['multiprocessing_context'] = 'forkserver'

    ##### HOROVOD #####
    train_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
    ##### HOROVOD #####
    # Horovod: use DistributedSampler to partition the training data.
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    ##### TODO:Need argument #####
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler, **kwargs)

    ##### HOROVOD #####
    test_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]))
    ##### HOROVOD #####
    # Horovod: use DistributedSampler to partition the test data.
    test_sampler = torch.utils.data.distributed.DistributedSampler(
        test_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    ##### TODO:Need argument #####
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                              sampler=test_sampler, **kwargs)

    model = Net()

    ##### HOROVOD #####
    # By default, Adasum doesn't need scaling up learning rate.
    lr_scaler = hvd.size() if not use_adasum else 1

    if cuda:
        # Move model to GPU.
        model.cuda()
        # If using GPU Adasum allreduce, scale learning rate by local_size.
        ##### TODO:Need argument #####
        if use_adasum and hvd.nccl_built():
            lr_scaler = hvd.local_size()

    # Horovod: scale learning rate by lr_scaler.
    ##### TODO:Need argument #####
    '''
    optimizer = optim.SGD(model.parameters(), lr=args.lr * lr_scaler,
                          momentum=args.momentum)'''
    optimizer = optim.SGD(model.parameters(), lr=lr * lr_scaler,
                          momentum=momentum)

    ##### HOROVOD #####
    # Horovod: broadcast parameters & optimizer state.
    hvd.broadcast_parameters(model.state_dict(), root_rank=0)
    hvd.broadcast_optimizer_state(optimizer, root_rank=0)

    ##### HOROVOD #####
    # Horovod: (optional) compression algorithm.
    # compression = hvd.Compression.fp16 if args.fp16_allreduce else hvd.Compression.none
    compression = hvd.Compression.none

    ##### HOROVOD #####
    # Horovod: wrap optimizer with DistributedOptimizer.
    optimizer = hvd.DistributedOptimizer(optimizer,
                                         named_parameters=model.named_parameters(),
                                         compression=compression,
                                         op=hvd.Adasum if use_adasum else hvd.Average)

    for epoch in range(1, epochs + 1):
        train(epoch)
        test()

/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319920
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.322078
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.313202
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.290503
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.284245
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.306247
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.284189
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.256639
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.296750
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.207612
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.199354
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.165684
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.165754
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.195857
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.052883
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.981519
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.971234
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.786817
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.905536
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.558711
Train Epoch: 1 [12800/60000 (

/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Train Epoch: 2 [0/60000 (0%)]	Loss: 0.463838
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.365127
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.518499
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.641464
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.583523
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.345165
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.648178
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.535414
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.528342
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.437272
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.406183
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.350120
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.419388
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.454681
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.348987
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.625952
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.512543
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.285007
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.416173
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.479310
Train Epoch: 2 [12800/60000 (

## horovod template python file

In [1]:
# %%writefile ./train_hvd.py

from __future__ import print_function
import argparse
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data.distributed
import horovod.torch as hvd

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

def train(epoch):
    model.train()
    ##### HOROVOD ##### --- train_sampler, optimizer wrapper
    # Horovod: set epoch to sampler for shuffling.
    train_sampler.set_epoch(epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            ##### HOROVOD ##### --- train_sampler
            # Horovod: use train_sampler to determine the number of examples in
            # this worker's partition.
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_sampler),
                100. * batch_idx / len(train_loader), loss.item()))

def metric_average(val, name):
    tensor = torch.tensor(val)
    ##### HOROVOD ##### -- allreduce (tensor average)
    avg_tensor = hvd.allreduce(tensor, name=name)    
    return avg_tensor.item()

def test():
    model.eval()
    test_loss = 0.
    test_accuracy = 0.
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        test_accuracy += pred.eq(target.data.view_as(pred)).cpu().float().sum()

    ##### HOROVOD #####
    # Horovod: use test_sampler to determine the number of examples in
    # this worker's partition.
    test_loss /= len(test_sampler)
    test_accuracy /= len(test_sampler)

    ##### HOROVOD #####
    # Horovod: average metric values across workers.
    test_loss = metric_average(test_loss, 'avg_loss')
    test_accuracy = metric_average(test_accuracy, 'avg_accuracy')

    # Horovod: print output only on first rank.
    ##### HOROVOD #####
    if hvd.rank() == 0:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(
            test_loss, 100. * test_accuracy))

if __name__ == '__main__':
    
    ##### TODO: This is arguments set #####
    ##### CUDA SETTING #####
    no_cuda = False
    cuda = not no_cuda and torch.cuda.is_available()
    #########################
    batch_size = 64
    test_batch_size = 128
    epochs = 10    
    momentum = 0.5
    lr = 0.01
    log_interval = 10
    # For Parallel/Distributed
    seed = 42
    use_adasum = False

    # Horovod: initialize library.
    ##### HOROVOD #####
    hvd.init()
    torch.manual_seed(seed)

    if cuda:
        # Horovod: pin GPU to local rank.
        ##### HOROVOD #####
        torch.cuda.set_device(hvd.local_rank())
        ##### TODO:Need argument #####
        #torch.cuda.manual_seed(args.seed)
        torch.cuda.manual_seed(seed)


    # Horovod: limit # of CPU threads to be used per worker.
    torch.set_num_threads(1)

    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
    # When supported, use 'forkserver' to spawn dataloader workers instead of 'fork' to prevent
    # issues with Infiniband implementations that are not fork-safe
    if (kwargs.get('num_workers', 0) > 0 and hasattr(mp, '_supports_context') and
            mp._supports_context and 'forkserver' in mp.get_all_start_methods()):
        kwargs['multiprocessing_context'] = 'forkserver'

    ##### HOROVOD #####
    train_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
    ##### HOROVOD #####
    # Horovod: use DistributedSampler to partition the training data.
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    ##### TODO:Need argument #####
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler, **kwargs)

    ##### HOROVOD #####
    test_dataset = \
        datasets.MNIST('data-%d' % hvd.rank(), train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]))
    ##### HOROVOD #####
    # Horovod: use DistributedSampler to partition the test data.
    test_sampler = torch.utils.data.distributed.DistributedSampler(
        test_dataset, num_replicas=hvd.size(), rank=hvd.rank())
    ##### TODO:Need argument #####
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                              sampler=test_sampler, **kwargs)

    model = Net()

    ##### HOROVOD #####
    # By default, Adasum doesn't need scaling up learning rate.
    lr_scaler = hvd.size() if not use_adasum else 1

    if cuda:
        # Move model to GPU.
        model.cuda()
        # If using GPU Adasum allreduce, scale learning rate by local_size.
        ##### TODO:Need argument #####
        if use_adasum and hvd.nccl_built():
            lr_scaler = hvd.local_size()

    # Horovod: scale learning rate by lr_scaler.
    ##### TODO:Need argument #####
    '''
    optimizer = optim.SGD(model.parameters(), lr=args.lr * lr_scaler,
                          momentum=args.momentum)'''
    optimizer = optim.SGD(model.parameters(), lr=lr * lr_scaler,
                          momentum=momentum)

    ##### HOROVOD #####
    # Horovod: broadcast parameters & optimizer state.
    hvd.broadcast_parameters(model.state_dict(), root_rank=0)
    hvd.broadcast_optimizer_state(optimizer, root_rank=0)

    ##### HOROVOD #####
    # Horovod: (optional) compression algorithm.
    # compression = hvd.Compression.fp16 if args.fp16_allreduce else hvd.Compression.none
    compression = hvd.Compression.none

    ##### HOROVOD #####
    # Horovod: wrap optimizer with DistributedOptimizer.
    optimizer = hvd.DistributedOptimizer(optimizer,
                                         named_parameters=model.named_parameters(),
                                         compression=compression,
                                         op=hvd.Adasum if use_adasum else hvd.Average)

    for epoch in range(1, epochs + 1):
        train(epoch)
        test()

/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319920
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.322078
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.313202
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.290503
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.284245
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.306247
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.284189
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.256639
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.296750
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.207612
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.199354
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.165684
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.165754
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.195857
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.052883
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.981519
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.971234
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.786816
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.905536
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.558711
Train Epoch: 1 [12800/60000 (

/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2119, Accuracy: 93.75%



/home/sky/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train Epoch: 2 [0/60000 (0%)]	Loss: 0.474405
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.354797
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.523345
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.637119
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.598192
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.347066
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.668222
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.545920
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.508464
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.437701
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.426616
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.345147
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.412464
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.438034
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.350392
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.626064
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.517584
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.293942
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.406349
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.478566
Train Epoch: 2 [12800/60000 (